In [10]:
import pandas as pd
from unidecode import unidecode
import re

In [8]:
df = pd.read_parquet('train.parquet')
df.head(5)

,razaosocial,nome_fantasia,parsed_response,cnpj,identificador_matriz_filial,data_situacao_cadastral,nome_cidade_exterior,data_inicio_atividade,uf,capital_social,porte,cnae_fiscal,descricao_cnae,nome_pais,nome_natureza_juridica,nome_motivo,nome_qualificacao,user_input
50222,MAGAZINE LUIZA S/A,MAGAZINE LUIZA,MAGAZINE L,47960950025388,filial,0,None,20050317,SP,1.380216e+10,demais,4713004,"Lojas de departamentos ou magazines, exceto lo...",None,Sociedade Anônima Aberta,SEM MOTIVO,Diretor,MAGAZINE L
99431,GP PNEUS LTDA,GP PNEUS,PNEUS GP,46378127006695,filial,20221209,None,20221209,PR,1.156500e+08,demais,4530705,Comércio a varejo de pneumáticos e câmaras-de-ar,None,Sociedade Empresária Limitada,SEM MOTIVO,Sócio-Administrador,PNEUS GP
240152,DISTRIBUIDORA DE MEDICAMENTOS SANTA CRUZ LTDA,SANTA CRUZ,SANTA CRUZ DISTRIBUIDORA,61940292004710,filial,20051103,None,19970624,RS,6.722510e+08,demais,4644301,Comércio atacadista de medicamentos e drogas d...,None,Sociedade Empresária Limitada,SEM MOTIVO,Administrador,SANTA CRUZ DISTRIBUIDORA
207820,DROGAL FARMACEUTICA LTDA,DROGAL JAGUARIUNA,DROGALL,54375647029451,filial,20220613,None,20220613,SP,5.000000e+06,demais,4771701,"Comércio varejista de produtos farmacêuticos, ...",None,Sociedade Empresária Limitada,SEM MOTIVO,Sócio-Administrador,DROGALL
68386,"ALLPARK EMPREENDIMENTOS, PARTICIPACOES E SERVI...",ESTAPAR,ESTAPAR BRASIL LTDA,60537263145644,filial,20240604,None,20240604,ES,6.456299e+08,demais,5223100,Estacionamento de veículos,None,Sociedade Anônima Aberta,SEM MOTIVO,Presidente,ESTAPAR BRASIL LTDA


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 255471 entries, 50222 to 146600
Data columns (total 18 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   razaosocial                  255471 non-null  object 
 1   nome_fantasia                255471 non-null  object 
 2   parsed_response              255471 non-null  object 
 3   cnpj                         255471 non-null  object 
 4   identificador_matriz_filial  255471 non-null  object 
 5   data_situacao_cadastral      255471 non-null  int64  
 6   nome_cidade_exterior         554 non-null     object 
 7   data_inicio_atividade        255471 non-null  int64  
 8   uf                           255471 non-null  object 
 9   capital_social               255471 non-null  float64
 10  porte                        254949 non-null  object 
 11  cnae_fiscal                  255471 non-null  int64  
 12  descricao_cnae               255471 non-null  object 
 13  

In [55]:
# Padrões pré-compilados para eficiência
separator_pattern = re.compile(r'[-.,;:()\[\]!?@#$%&*_+=/\\|]+')
punctuation_pattern = re.compile(r'[^\w\s]')  # Mantém alfanuméricos e espaços
accent_pattern = re.compile(r'[\u0300-\u036f]')  # Unicode diacritic marks
space_pattern = re.compile(r'\s+')


def remove_juridical_terms(text, additional_stopwords=None):
    """
    Remove termos jurídicos e palavras irrelevantes de nomes de empresas
    
    Args:
        text: Texto pré-processado
        additional_stopwords: Lista de stopwords adicionais para remover
        
    Returns:
        Texto limpo
    """
    # Termos jurídicos básicos (em minúsculas, sem acentos)
    juridical_terms = {
        'sociedade', 'anonima', 'limitada', 'ltda', 'me', 'epp', 'ep', 
        'sa', 's a', 'comercio', 'industria', 'servicos', 'empresa',
        'falido', 'falencia'
    }
    
    # Adicionar stopwords customizadas se fornecidas
    if additional_stopwords:
        juridical_terms |= set(additional_stopwords)
    
    # Dividir o texto em palavras e filtrar
    words = [word for word in text.split() if word not in juridical_terms]
    
    return ' '.join(words).strip()



def simplify_text(text, keep_accents=False, remove_juridical=True, additional_stopwords=None):
    if not isinstance(text, str) or not text.strip():
        return ""
    
    
    # Etapa 1: Converter para minúsculas
    text = text.lower()
    
    # Etapa 1.5: Tratamento especial para S.A. e S/A
    text = re.sub(r'\b(s\.?\s*[aàáâã]\.?|s\s*/\s*a)\b', 'sa', text)
    
    # Etapa 2: Normalizar separadores - converter para espaços
    text = separator_pattern.sub(' ', text)
    
    # Etapa 3: Tratamento de acentos e caracteres especiais
    if not keep_accents:
        text = unidecode(text)  # Remove acentos e normaliza caracteres
        text = punctuation_pattern.sub('', text)  # Remove pontuação
    else:
        # Remove apenas marcas diacríticas mantendo caracteres base
        text = accent_pattern.sub('', text)
        text = punctuation_pattern.sub('', text)  # Remove pontuação
    
    # Etapa 4: Normalizar espaços (remove duplos, mantém únicos)
    text = space_pattern.sub(' ', text).strip()
    
    # Etapa 5: Substituir termos jurídicos por extenso
    # Usando regex com \b para garantir palavras completas
    text = re.sub(r'\bsa\b', 'sociedade anonima', text)
    text = re.sub(r'\bltda\b', 'limitada', text)
    
    if remove_juridical:
        text = remove_juridical_terms(text, additional_stopwords)
    
    # Normalizar espaços novamente após substituições
    text = space_pattern.sub(' ', text).strip()
    
    return text
def process_dataframe(df, columns, keep_accents=False):
    """
    Process the DataFrame by applying text simplification to specified columns.
    
    Args:
        df (pd.DataFrame): The DataFrame to process.
        columns (list): List of column names to apply text simplification.
    
    Returns:
        pd.DataFrame: The processed DataFrame with simplified text in specified columns.
    """
    df = df.copy()  # Avoid modifying the original DataFrame
    
    for col in columns:
        new_col = f"{col}_clean"
        df[new_col] = df[col].apply(lambda x: simplify_text(x, keep_accents))
    return df

In [37]:
text_columns = ['razaosocial', 'nome_fantasia', 'parsed_response', 'uf']

In [56]:
df_clean = process_dataframe(df, text_columns)
df_clean.head(50)

,razaosocial,nome_fantasia,parsed_response,cnpj,identificador_matriz_filial,data_situacao_cadastral,nome_cidade_exterior,data_inicio_atividade,uf,capital_social,...,descricao_cnae,nome_pais,nome_natureza_juridica,nome_motivo,nome_qualificacao,user_input,razaosocial_clean,nome_fantasia_clean,parsed_response_clean,uf_clean
50222,MAGAZINE LUIZA S/A,MAGAZINE LUIZA,MAGAZINE L,47960950025388,filial,0,None,20050317,SP,1.380216e+10,...,"Lojas de departamentos ou magazines, exceto lo...",None,Sociedade Anônima Aberta,SEM MOTIVO,Diretor,MAGAZINE L,magazine luiza,magazine luiza,magazine l,sp
99431,GP PNEUS LTDA,GP PNEUS,PNEUS GP,46378127006695,filial,20221209,None,20221209,PR,1.156500e+08,...,Comércio a varejo de pneumáticos e câmaras-de-ar,None,Sociedade Empresária Limitada,SEM MOTIVO,Sócio-Administrador,PNEUS GP,gp pneus,gp pneus,pneus gp,pr
240152,DISTRIBUIDORA DE MEDICAMENTOS SANTA CRUZ LTDA,SANTA CRUZ,SANTA CRUZ DISTRIBUIDORA,61940292004710,filial,20051103,None,19970624,RS,6.722510e+08,...,Comércio atacadista de medicamentos e drogas d...,None,Sociedade Empresária Limitada,SEM MOTIVO,Administrador,SANTA CRUZ DISTRIBUIDORA,distribuidora de medicamentos santa cruz,santa cruz,santa cruz distribuidora,rs
207820,DROGAL FARMACEUTICA LTDA,DROGAL JAGUARIUNA,DROGALL,54375647029451,filial,20220613,None,20220613,SP,5.000000e+06,...,"Comércio varejista de produtos farmacêuticos, ...",None,Sociedade Empresária Limitada,SEM MOTIVO,Sócio-Administrador,DROGALL,drogal farmaceutica,drogal jaguariuna,drogall,sp
68386,"ALLPARK EMPREENDIMENTOS, PARTICIPACOES E SERVI...",ESTAPAR,ESTAPAR BRASIL LTDA,60537263145644,filial,20240604,None,20240604,ES,6.456299e+08,...,Estacionamento de veículos,None,Sociedade Anônima Aberta,SEM MOTIVO,Presidente,ESTAPAR BRASIL LTDA,allpark empreendimentos participacoes e,estapar,estapar brasil,es
291843,LRE UNIVERSO DAS TINTAS LTDA,UNIVERSO DAS TINTAS,UNIV DAS TINTAS,42036510000169,matriz,20210521,None,20210521,MA,2.000000e+04,...,Comércio varejista de tintas e materiais para ...,None,Sociedade Empresária Limitada,SEM MOTIVO,Sócio-Administrador,UNIB DAS TINTAS,lre universo das tintas,universo das tintas,univ das tintas,ma
239942,SEST SERVICO SOCIAL DO TRANSPORTE,FOZ DO IGUACU/PR - OCTAVIANO DA ROS - UNIDADE ...,IGUAÇU UNIDADE,73471989004778,filial,19980728,None,19960916,PR,0.000000e+00,...,Serviços de assistência social sem alojamento,None,Serviço Social Autônomo,SEM MOTIVO,Administrador,IGUACU UNIDADE,sest servico social do transporte,foz do iguacu pr octaviano da ros unidade b n 086,iguacu unidade,pr
253364,COASUL COOPERATIVA AGROINDUSTRIAL,ENTREPOSTO - FRANCISCO BELTRAO - GAUCHA,FRANCISCO BELTRAO,79863569005442,filial,20191009,None,20191009,PR,0.000000e+00,...,Comércio atacadista de matérias-primas agrícol...,None,Cooperativa,SEM MOTIVO,Presidente,FRANSISCO BELTRAO,coasul cooperativa agroindustrial,entreposto francisco beltrao gaucha,francisco beltrao,pr
226262,DISTRIBUIDORA CURITIBA DE PAPEIS E LIVROS S/A,LIVRARIAS CURITIBA,CURITIBA DISTRIBUIDORA,79065181004696,filial,20220117,None,20220117,SP,5.461462e+07,...,Comércio varejista de livros,None,Sociedade Anônima Fechada,SEM MOTIVO,Diretor,CURITIBA DISTRIBUIDORA,distribuidora curitiba de papeis e livros,livrarias curitiba,curitiba distribuidora,sp
33041,CONGREGACAO CRISTA NO BRASIL,CONGREGACAO CRISTA NO BRASIL,CONGREGACAO BRASIL CRISTA,44252674000825,filial,20180528,None,20180528,SP,0.000000e+00,...,Atividades de organizações religiosas ou filos...,None,Organização Religiosa,SEM MOTIVO,Presidente,CONGREGACAO BRASIL CRISTA,congregacao crista no brasil,congregacao crista no brasil,congregacao brasil crista,sp
